# Writing Deschutes NF multi-objective CPLEX model file

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Define object that is a body of all req'd data for the model file
class DeschutesModelData:
    def __init__(self, areas, fire, sed, clusters, nsoCands,
                 maxPerPeriodTrt, interPeriodFluctuation, nsoHabSizeDiscount):
        self.areas = areas
        self.stands = areas["stand"]
        self.fire = fire
        self.climProjections = fire["climateScenario"].unique().tolist()
        self.sed = sed
        self.clusters = clusters
        self.nsoCands = nsoCands
        self.maxPerPeriodTrt = maxPerPeriodTrt
        self.interPeriodFluctuation = interPeriodFluctuation
        self.nsoHabSizeDiscount = nsoHabSizeDiscount

In [3]:
# Instantiate a DeschutesModelData object with our data files
dmd = DeschutesModelData(areas = pd.read_csv("data/standAreas_test.csv"),
                         fire = pd.read_csv("data/fire_test.csv"),
                         sed = pd.read_csv("data/sed_wClim_test.csv"),
                         clusters = pd.read_csv("data/clusters_test.csv"),
                         nsoCands = pd.read_csv("data/standNSOCandidacy_wTrtTimes_test.csv"),
                         maxPerPeriodTrt = 6000,
                         interPeriodFluctuation = 0.2,
                         nsoHabSizeDiscount = 0.5)

In [4]:
# Create method to write the constraint defining the fire objective
def writeFireObjDefinition(model, fireTable):
    model.write("FireObjective: ")
    for index, row in fireTable.iterrows():
        s = str(int(row.stand))
        for idx, val in enumerate(row["trtInNeither":]):
            if val < 0:
                model.write(" " + str(val) + " x_" + s + "_" + str(idx))
            else:
                model.write(" + " + str(val) + " x_" + s + "_" + str(idx))
        model.write("\n")
    model.write(" - FireHazardReduction = 0\n")

In [10]:
# Create method to write the constraint defining the owl objective
def writeOwlObjDefinition(model, owlTable, areas, discount):
    owl = owlTable.merge(areas, how="left", on="stand")
    owl = owl[["stand","area","trtInNeither","trtIn1","trtIn2","trtInBoth"]]
    model.write("OwlObjective: ")
    for index, row in owl.iterrows():
        s = str(int(row.stand))
        a = row.area
        if sum(row["trtInNeither":]) > 0: # If the stand qualifies as NSO habitat in at least one treatment scenario...
            model.write(str(a) + " p_" + s + " - " + str(discount*a) + " p_" + s) # Then the site may contribute to the objective function
            for idx, val in enumerate(row["trtInNeither":]): # So we sum up over treatment scenarios...
                if val > 0: # in which the stand qualifies for NSO habitat (val will be 1)
                    model.write(" + " + str(discount*a) + " x_" + s + "_" + str(idx))
            model.write("\n")
    model.write(" - OwlHabitat = 0\n")
                

In [7]:
# Umbrella methods for the writing of the CPLEX model file
def writeObjective(model):
    model.write("MAXIMIZE\n")
    model.write("OBJECTIVE: FireHazardReduction + OwlHabitat - MaxSediment\n\n")

def writeConstraints(model, clim, dmd):
    model.write("Subject To:\n")
    writeFireObjDefinition(model, dmd.fire.loc[dmd.fire.climateScenario == clim])
    writeOwlObjDefinition(model, dmd.nsoCands.loc[dmd.nsoCands.climateScenario == clim], dmd.areas, dmd.nsoHabSizeDiscount)
    #writeSedimentObjDefinition(model, dmd)
    #writeClusterVarTriggers(model, dmd) # the per-cluster constraints (eq 5 in SEFS 540 final report)
    #writeOwlClusterSiteTriggers(model, dmd) # the per-site constraints on the value of p_i
    #writeOnePrescripPerSite(model, dmd) # per-site, sum of prescriptions is 1
    #writeMaxAreaTreatedThresh(model, dmd) # per time period, sum of area treated < maxPerPeriodTrt
    #writeTreatedAreaFluctThresh(model, dmd) # difference in period to period treatment areas
    model.write("\n")
    
def writeVariableStatement(model, dmd):
    model.write("BINARY\n")
    #writeBinaryVarStmt(model, dmd)
    model.write("\n")

In [11]:
# Write set of CPLEX model files

for clim in dmd.climProjections:
    with open("modelFiles/testModel_" + clim + ".txt", "w") as model:
        model.write("\\ Deschutes NF model file for climate scenario " + clim + "\n")
        writeObjective(model)
        writeConstraints(model, clim, dmd)
        writeVariableStatement(model, dmd)
        model.write("\nEND\n")
    model.closed